# 03 — SPC: Shewhart & CUSUM
Significance: catch sudden and persistent drifts in weekly margin%.


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('../data/service_orders_synthetic.csv', parse_dates=['order_dt'])
weekly = (df.groupby([df['order_dt'].dt.isocalendar().week], as_index=False)
            .agg(mean_margin=('margin_pct','mean')))
weekly.columns=['week','mean_margin']
m, s = weekly['mean_margin'].mean(), weekly['mean_margin'].std()
ucl, lcl = m+3*s, m-3*s
plt.figure(); plt.plot(weekly['week'], weekly['mean_margin'], marker='o');
plt.axhline(m, linestyle='--'); plt.axhline(ucl, linestyle='--'); plt.axhline(lcl, linestyle='--');
plt.title('Shewhart Chart: Weekly Mean Margin%'); plt.show()

In [ ]:
k = 0.5*s; h = 5*s; pos=[0]; neg=[0]; signals=[]
for i, x in enumerate(weekly['mean_margin']):
    pos.append(max(0, pos[-1] + (x - (m + k))))
    neg.append(min(0, neg[-1] + (x - (m - k))))
    if pos[-1] > h or neg[-1] < -h: signals.append(i)
plt.figure(); plt.plot(pos[1:], label='CUSUM+'); plt.plot(neg[1:], label='CUSUM-'); plt.legend(); plt.title('CUSUM'); plt.show(); signals